In [1]:
import psycopg2
import pandas.io.sql as sqlio
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Conexão com BD

In [4]:
user = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'
database = 'ANP'
table_name = 'preco_combustivel'
schema_name = 'anp'

In [5]:
conn = psycopg2.connect(dbname=database, user=user, password=password,
                        host=host, port=port)

# Consultas básicas

In [6]:
sql = f"""
SELECT * FROM {schema_name}.{table_name}
"""

df = sqlio.read_sql_query(sql, conn)

In [7]:
df.head()

,regiao,uf,municipio,revenda,cnpj,nome_rua,numero_rua,complemento,bairro,cep,produto,data_coleta,valor_venda,unidade_medida,bandeira
0,NE,BA,SALVADOR,PETROBRAS DISTRIBUIDORA S.A.,34.274.233/0015-08,RUA EDISTIO PONDE,474,None,STIEP,41770-395,GNV,2018-01-02,2.370,R$ / m³,PETROBRAS DISTRIBUIDORA S.A.
1,NE,BA,SALVADOR,PETROBRAS DISTRIBUIDORA S.A.,34.274.233/0015-08,RUA EDISTIO PONDE,474,None,STIEP,41770-395,DIESEL S10,2018-01-02,3.240,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
2,NE,BA,SALVADOR,PETROBRAS DISTRIBUIDORA S.A.,34.274.233/0015-08,RUA EDISTIO PONDE,474,None,STIEP,41770-395,ETANOL,2018-01-02,2.930,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
3,NE,BA,SALVADOR,PETROBRAS DISTRIBUIDORA S.A.,34.274.233/0015-08,RUA EDISTIO PONDE,474,None,STIEP,41770-395,GASOLINA,2018-01-02,3.620,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
4,S,RS,CANOAS,METROPOLITANO COMERCIO DE COMBUSTIVEIS LTDA,88.587.589/0001-17,AVENIDA GUILHERME SCHELL,6340,None,CENTRO,92310-000,GNV,2018-01-02,2.699,R$ / m³,BRANCA


In [8]:
print(df.shape)

(4402370, 15)


# Tipos de dados

In [9]:
print(df.dtypes)

regiao                    object
uf                        object
municipio                 object
revenda                   object
cnpj                      object
nome_rua                  object
numero_rua                object
complemento               object
bairro                    object
cep                       object
produto                   object
data_coleta       datetime64[ns]
valor_venda              float64
unidade_medida            object
bandeira                  object
dtype: object


# Dados nulos

In [10]:
print(df.isna().sum())

regiao              0
uf                  0
municipio           0
revenda             0
cnpj                0
nome_rua            0
numero_rua          6
complemento       612
bairro              0
cep                 0
produto             0
data_coleta         0
valor_venda         0
unidade_medida      0
bandeira            0
dtype: int64


# Organização das colunas

In [11]:
df_anp = df[['data_coleta','regiao','uf','municipio','bairro','bandeira','produto','valor_venda','unidade_medida']]
df_anp.head()

,data_coleta,regiao,uf,municipio,bairro,bandeira,produto,valor_venda,unidade_medida
0,2018-01-02,NE,BA,SALVADOR,STIEP,PETROBRAS DISTRIBUIDORA S.A.,GNV,2.370,R$ / m³
1,2018-01-02,NE,BA,SALVADOR,STIEP,PETROBRAS DISTRIBUIDORA S.A.,DIESEL S10,3.240,R$ / litro
2,2018-01-02,NE,BA,SALVADOR,STIEP,PETROBRAS DISTRIBUIDORA S.A.,ETANOL,2.930,R$ / litro
3,2018-01-02,NE,BA,SALVADOR,STIEP,PETROBRAS DISTRIBUIDORA S.A.,GASOLINA,3.620,R$ / litro
4,2018-01-02,S,RS,CANOAS,CENTRO,BRANCA,GNV,2.699,R$ / m³


# Criar colunas de ano e mês

In [12]:
df_anp['ano'] = df_anp['data_coleta'].dt.year
df_anp['mes'] = df_anp['data_coleta'].dt.month

df_anp

,data_coleta,regiao,uf,municipio,bairro,bandeira,produto,valor_venda,unidade_medida,ano,mes
0,2018-01-02,NE,BA,SALVADOR,STIEP,PETROBRAS DISTRIBUIDORA S.A.,GNV,2.370,R$ / m³,2018,1
1,2018-01-02,NE,BA,SALVADOR,STIEP,PETROBRAS DISTRIBUIDORA S.A.,DIESEL S10,3.240,R$ / litro,2018,1
2,2018-01-02,NE,BA,SALVADOR,STIEP,PETROBRAS DISTRIBUIDORA S.A.,ETANOL,2.930,R$ / litro,2018,1
3,2018-01-02,NE,BA,SALVADOR,STIEP,PETROBRAS DISTRIBUIDORA S.A.,GASOLINA,3.620,R$ / litro,2018,1
4,2018-01-02,S,RS,CANOAS,CENTRO,BRANCA,GNV,2.699,R$ / m³,2018,1
...,...,...,...,...,...,...,...,...,...,...,...
4402365,2022-12-30,S,SC,BLUMENAU,ESCOLA AGRICOLA,MAXSUL,GASOLINA,5.040,R$ / litro,2022,12
4402366,2022-12-30,S,SC,BLUMENAU,ESCOLA AGRICOLA,MAXSUL,DIESEL S10,6.190,R$ / litro,2022,12
4402367,2022-12-30,S,SC,BLUMENAU,NOVA ESPERANCA,VIBRA ENERGIA,GASOLINA ADITIVADA,5.310,R$ / litro,2022,12
4402368,2022-12-30,S,SC,BLUMENAU,NOVA ESPERANCA,VIBRA ENERGIA,GASOLINA,5.110,R$ / litro,2022,12


# Estatística básica

In [14]:
df_anp.describe().round(2)

,data_coleta,valor_venda,ano,mes
count,4402370,4402370.00,4402370.00,4402370.00
mean,2020-05-28 23:53:03.226671616,4.48,2019.93,6.27
min,2018-01-01 00:00:00,1.76,2018.00,1.00
25%,2019-02-18 00:00:00,3.52,2019.00,3.00
50%,2020-03-25 00:00:00,4.19,2020.00,6.00
75%,2021-10-20 00:00:00,5.20,2021.00,9.00
max,2022-12-30 00:00:00,9.65,2022.00,12.00
std,NaN,1.30,1.45,3.38


In [15]:
df_anp.query('valor_venda == @df_anp.valor_venda.max()')

,data_coleta,regiao,uf,municipio,bairro,bandeira,produto,valor_venda,unidade_medida,ano,mes
4395335,2022-12-27,SE,SP,SAO PAULO,JARDIM SANTA FE,IPIRANGA,DIESEL S10,9.65,R$ / litro,2022,12


# Perguntas para os dados

## Quais produtos foram comercializados?

In [16]:
print(df_anp.produto.unique())

['GNV' 'DIESEL S10' 'ETANOL' 'GASOLINA' 'DIESEL' 'GASOLINA ADITIVADA']


## Quais anos estão na base?

In [17]:
print(df_anp.ano.unique())

[2018 2019 2020 2021 2022]


## Quais os valores mínimos, médios e máximos por ano para cada produto?

In [29]:
min_max_produto_ano = df_anp[['produto','ano','valor_venda']].groupby(['produto','ano']).agg(['min','max','mean']).round(2).reset_index()
min_max_produto_ano

produto   ano valor_venda            
                                     min   max  mean
0               DIESEL  2018        2.80  5.20  3.50
1               DIESEL  2019        2.87  4.99  3.60
2               DIESEL  2020        2.45  5.09  3.42
3               DIESEL  2021        3.10  6.99  4.69
4               DIESEL  2022        3.14  9.00  6.63
5           DIESEL S10  2018        2.84  5.30  3.59
6           DIESEL S10  2019        2.79  5.09  3.69
7           DIESEL S10  2020        2.46  5.38  3.51
8           DIESEL S10  2021        2.80  6.96  4.74
9           DIESEL S10  2022        3.59  9.65  6.73
10              ETANOL  2018        1.90  4.98  3.13
11              ETANOL  2019        2.10  5.47  3.17
12              ETANOL  2020        1.80  5.15  3.18
13              ETANOL  2021        2.05  7.90  4.67
14              ETANOL  2022        2.49  7.98  4.85
15            GASOLINA  2018        3.37  6.29  4.45
16            GASOLINA  2019        3.39  6.29  4.42
17            GASOLINA  2020        2.87  5.90  4.28
18            GASOLINA  2021        3.10  8.00  5.89
19            GASOLINA  2022        3.49  8.99  6.32
20  GASOLINA ADITIVADA  2020        3.69  6.00  4.59
21  GASOLINA ADITIVADA  2021        3.46  8.99  6.04
22  GASOLINA ADITIVADA  2022        4.09  9.28  6.47
23                 GNV  2018        1.76  4.10  2.73
24                 GNV  2019        2.00  4.56  3.22
25                 GNV  2020        2.00  4.78  3.17
26                 GNV  2021        2.38  6.70  3.89
27                 GNV  2022        3.17  7.99  5.04

## Quais os valores mínimos, médios e máximos por estado e ano para cada produto?

In [33]:
min_max_ano_estado = df_anp[['ano','produto','uf','valor_venda']].groupby(['ano','produto','uf']).agg(['min','max','mean']).round(2).reset_index()
min_max_ano_estado

ano produto  uf valor_venda            
                              min   max  mean
0    2018  DIESEL  AC        3.38  4.89  4.29
1    2018  DIESEL  AL        3.10  4.23  3.60
2    2018  DIESEL  AM        3.09  4.49  3.63
3    2018  DIESEL  AP        3.29  4.51  3.88
4    2018  DIESEL  BA        2.99  5.20  3.51
..    ...     ...  ..         ...   ...   ...
722  2022     GNV  RN        4.11  4.99  4.50
723  2022     GNV  RS        4.59  6.89  5.88
724  2022     GNV  SC        4.00  6.89  5.50
725  2022     GNV  SE        4.47  5.49  4.99
726  2022     GNV  SP        3.59  6.99  5.22

[727 rows x 6 columns]

## Qual a quantidade de bandeiras comercializando cada produto em cada ano?

In [36]:
df_anp_bandeira = df_anp[['ano','bandeira','produto']].groupby(['ano','produto']).bandeira.nunique()
df_anp_bandeira

ano   produto           
2018  DIESEL                64
      DIESEL S10            66
      ETANOL                66
      GASOLINA              66
      GNV                   24
2019  DIESEL                59
      DIESEL S10            61
      ETANOL                63
      GASOLINA              64
      GNV                   22
2020  DIESEL                52
      DIESEL S10            57
      ETANOL                58
      GASOLINA              58
      GASOLINA ADITIVADA    43
      GNV                   23
2021  DIESEL                54
      DIESEL S10            54
      ETANOL                55
      GASOLINA              55
      GASOLINA ADITIVADA    54
      GNV                   22
2022  DIESEL                51
      DIESEL S10            60
      ETANOL                60
      GASOLINA              60
      GASOLINA ADITIVADA    58
      GNV                   24
Name: bandeira, dtype: int64

## Qual a quantidade de bandeiras comercializando cada produto em cada ano e em cada estado?

In [38]:
df_anp_bandeira_estado = df_anp[['ano','uf','bandeira','produto']].groupby(['ano','produto','uf']).bandeira.nunique()
df_anp_bandeira_estado

ano   produto  uf
2018  DIESEL   AC     6
               AL     7
               AM     9
               AP     3
               BA    10
                     ..
2022  GNV      RN     5
               RS     6
               SC     8
               SE     5
               SP     7
Name: bandeira, Length: 727, dtype: int64